<a href="https://colab.research.google.com/github/Mukool-Jadhav/MyProjects/blob/main/PHI3_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers accelerate bitsandbytes

In [2]:
!pip install sentence-transformers

In [5]:
pip install flash_attn

In [6]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoProcessor

def get_image_analysis(prompt, image_path, model_id="microsoft/Phi-3-vision-128k-instruct"):
    """
    This function analyzes an image and generates a response based on a prompt using a large language model.

    Args:
        prompt (str): The text prompt for the LLM model.
        image_path (str): The file path to the image to analyze.
        model_id (str): The identifier of the LLM model to be used (defaults to Phi-3).

    Returns:
        str: The generated response by the LLM model.
    """

    # Load model and processor based on the model_id
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation="eager")
    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

    # Prepare the conversation history
    messages = [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "Description about entities"},
        {"role": "user", "content": "Task is to summarize about the given data based on the extracted entites"}
    ]

    # Open the image

    image = Image.open(requests.get(image_path, stream=True).raw)


    # Apply the chat template
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

    # Generation arguments (you can adjust these as needed)
    generation_args = {
        "max_new_tokens": 500,
        "temperature": 0.0,
        "do_sample": False,
    }

    # Generate response using the model
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

    # Remove input tokens from the generated IDs
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]

    # Decode the generated IDs back to text
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    return response




In [7]:
# Example usage
prompt = "<|image_1|>\nYour Task is to extract entities"
image_path = "https://www.blueshieldca.com/bsca/code-assets/images/sample-eob_popup.jpg"
response = get_image_analysis(prompt, image_path)
print(response)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Summary of the given data based on the extracted entities is as follows:

1. Patient Name: TAYLOR SMITH
2. Subscriber ID: J23456789-0000
3. Claim Number: 12345678900000
4. Patient responsibility: $38.25
5. Amount we paid: $74.00
6. Network savings: $15.75
7. Amount billed by Provider: $128.00
8. Deductible Status: As of 06/06/04, TAYLOR SMITH has met $38.25 of the $500.00 annual deductible for 2004.
9. Provider: LABORATORY CORP OF AMERICA HOL
10. Preferred Provider: YES
11. Service Date: 05/08/04
12. Type of Service and Procedure Number: OUTPTLABORATORY 80053
13. Amount Billed: 35.00
14. Amount Allowed: 35.00
15. Amount Paid: 35.00
16. Non Covered: 0.00
17. Deductible: 0.00
18. Copayment/ Coinsurance: 0.00
19. Notes: The provider of service has agreed to accept the allowed amount as payment in full. The subscriber is responsible only for deductibles, copayment amounts and non covered items.
20. Messages: If your plan requires hospital pre-admission review, you or your physician must co